# Lab : Text-to-Speech avec Tacotron2 et HiFi-GAN

Ce laboratoire vous guide pas à pas pour passer d’un texte brut à un fichier audio `.wav` en utilisant Tacotron2 pour générer les spectrogrammes Mel, puis HiFi-GAN pour la synthèse vocale de haute qualité.

## 1. Préparation de l’environnement

> **But** : cloner le dépôt Tacotron2-PyTorch, Hifi-GAN et installer toutes les dépendances Python nécessaires.

In [ ]:
!git clone https://github.com/BogiHsu/Tacotron2-PyTorch.git
!pip install -r Tacotron2-PyTorch/requirements.txt
!git clone https://github.com/jik876/hifi-gan.git
!mkdir mel_files

## 2. Téléchargement du modèle Tacotron2 pré-entraîné

> **But** : récupérer le fichier de checkpoint `.pt` contenant les poids entraînés sur LJSpeech.

In [ ]:
import urllib.request
import os

url = "https://github.com/BogiHsu/Tacotron2-PyTorch/releases/download/lj-200k-b512/ckpt_200000"
out_path = os.path.join("Tacotron2-PyTorch", "ckpt_200000.pt")

# Crée le dossier s'il n'existe pas
os.makedirs(os.path.dirname(out_path), exist_ok=True)

# Téléchargement
print(f"Téléchargement de {url} → {out_path} …")
urllib.request.urlretrieve(url, out_path)
print("Terminé.")

## 3. Téléchargement du modèle HiFi-GAN pré-entraîné

> **But** : récupérer la configuration et les poids du vocodeur HiFi-GAN.

In [ ]:
import gdown

# 1) Télécharger le fichier config.json de HiFi-GAN
# 2) Télécharger le checkpoint du générateur

dl = {'hifi-gan/config.json': 'https://drive.google.com/u/1/uc?id=1aDh576AEYA5eTjhx7sew1qcCM_Y526jc&export=download',
      'hifi-gan/generator_v1': 'https://drive.google.com/u/1/uc?id=14NENd4equCBLyyCSke114Mv6YR_j_uFs&export=download'}
for k in dl:
    gdown.download(dl[k], k)

## 4. Inference Tacotron2

> **But** : générer le spectrogramme Mel à partir d’un texte donné.

In [ ]:
%cd Tacotron2-PyTorch
! "" > utils/__init__.py

!python inference.py \
--ckpt_pth=ckpt_200000.pt \
--text="Hello" \
--npy_pth=../mel_files/output_mel

## 5. Visualisation du spectrogramme Mel

> **But** : afficher le spectrogramme pour vérifier la qualité de la génération.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Charger le spectrogramme
mel = np.load('../mel_files/output_mel.npy')

# 2) Afficher
plt.figure(figsize=(12, 4))
plt.imshow(mel.T, aspect='auto', origin='lower')
plt.title("Spectrogramme Mel généré par Tacotron2")
plt.xlabel("Frame")
plt.ylabel("Canaux Mel")
plt.colorbar()
plt.show()

## 6. Synthèse audio avec HiFi-GAN

> **But** : convertir le spectrogramme Mel en onde audio et enregistrer un `.wav`.

In [ ]:
%cd hifi-gan
!python inference_e2e.py --checkpoint_file generator_v1 --input_mels_dir ../mel_files --output_dir ../wav_files
%cd ../

## 7. Lecture du résultat

> **But** : écouter directement le fichier généré dans le notebook.

In [ ]:
import IPython
IPython.display.Audio('wav_files/output_mel_generated_e2e.wav')